# Classification and regression models

In [1]:
import rebound
from spock import FeatureClassifier
feature_model = FeatureClassifier()

sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1., e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.move_to_com()

print(feature_model.predict_stable(sim))

0.06591137


In [2]:
from spock import DeepRegressor
deep_model = DeepRegressor()

median, lower, upper = deep_model.predict_instability_time(sim, samples=10000)
print(int(median))

236297


In [3]:
from spock import CollisionMergerClassifier
class_model = CollisionMergerClassifier()

prob_12, prob_23, prob_13 = class_model.predict_collision_probs(sim)

print('Probability planet 1 collides with 2:', prob_12)
print('Probability planet 2 collides with 3:', prob_23)
print('Probability planet 1 collides with 3:', prob_13)

Probability planet 1 collides with 2: 0.2738345
Probability planet 2 collides with 3: 0.49277353
Probability planet 1 collides with 3: 0.23339202


In [4]:
from spock import CollisionOrbitalOutcomeRegressor
reg_model = CollisionOrbitalOutcomeRegressor()

new_sim = reg_model.predict_collision_outcome(sim, collision_inds=[2, 3])

print('Predicted P1:', new_sim.particles[1].P)
print('Predicted e1:', new_sim.particles[1].e, '\n')
print('Predicted P2:', new_sim.particles[2].P)
print('Predicted e2:', new_sim.particles[2].e)

Predicted P1: 1.225037672556297
Predicted e1: 0.05888948547676769 

Predicted P2: 1.412646240582396
Predicted e2: 0.02929837424331723


# Giant impact emulator

In [5]:
import rebound
sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1, e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.add(m=1.e-5, P=2.2, e=0.03, pomega=0.5, l=5.0)
sim.add(m=1.e-5, P=2.5, e=0.03, pomega=5.0, l=1.5)
sim.move_to_com()

In [6]:
from spock import GiantImpactPhaseEmulator
emulator = GiantImpactPhaseEmulator(seed=42)

In [7]:
%%time
pred_sim = emulator.predict(sim)

CPU times: user 3.79 s, sys: 10.3 s, total: 14.1 s
Wall time: 3.6 s


In [8]:
ps = pred_sim.particles
print('t =', pred_sim.t, '\n')
for i in range(1, len(ps)):
    print('Predicted m' + str(i), '=', ps[i].m)
    print('Predicted P' + str(i), '=', ps[i].P)
    print('Predicted e' + str(i), '=', ps[i].e, '\n')

t = 999999999.9999993 

Predicted m1 = 2e-05
Predicted P1 = 1.1423167995914096
Predicted e1 = 0.024555185595986646 

Predicted m2 = 1e-05
Predicted P2 = 2.241317603301388
Predicted e2 = 0.028255655743078135 

Predicted m3 = 2e-05
Predicted P3 = 3.514830496498667
Predicted e3 = 0.0180188346495417 

